# Apnea Audio Data Preparation Pipeline

This notebook provides a complete and robust pipeline for preparing audio data for apnea event detection. It performs the following steps for each patient specified in the configuration:

1.  **Downloads** EDF (audio) and RML (event annotation) files from provided URLs.
2.  **Validates** that all of a patient's audio files have the same sample rate to prevent data corruption.
3.  **Extracts** apnea event timestamps (`Obstructive`, `Central`, `Mixed`) from the RML file.
4.  **Concatenates** the 'Mic' channel from all of a patient's EDF files into a single, continuous WAV file.
5.  **Analyzes** the WAV file second-by-second, extracting a rich set of 23 audio features for each second.
6.  **Labels** each second as `1` (apnea) or `0` (normal) based on the extracted event timestamps.
7.  **Appends** the features and labels for each patient into a single master CSV file, ready for model training.

In [4]:
!pip install mne soundfile

In [5]:
# === 1. IMPORTS ===
# --- Standard Libraries ---
import os
import csv
import xml.etree.ElementTree as ET

# --- Data and Signal Processing ---
import numpy as np
import scipy.stats
import librosa
import soundfile as sf
import mne

# --- File Downloading ---
import requests

In [6]:
# === 2. CONFIGURATION ===
# --- Edit this section for your patients ---

# List of patient data. Each dictionary represents one patient.
# Add a new dictionary to this list for each new patient you want to process.
patient_groups = [
    {
        "rml_url": "https://china.scidb.cn/download?fileId=644b5b677869a339f571e4a6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ef&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957", "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ee&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957","https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ed&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"
        ,"https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ec&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957","https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1eb&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6437f9c27869a339f571d95b&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ea&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957", "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1e9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1e8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957","https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1e7&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1e6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=644b5b677869a339f571e4a8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf367&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957", "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf369&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957", "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf366&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf36a&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957","https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf365&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=644b5b677869a339f571e4a9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1e0&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1df&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1de&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1dd&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6437f9c39870236416bbf9be&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1dc&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1db&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1da&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"
                     ,"https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6437f9c27869a339f571d95d&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d7&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d5&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1d4&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722e9870236416bbf69f&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ce&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1cd&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1cc&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1cb&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6437f9c39870236416bbf9bf&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ca&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c7&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c5&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c4&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722e7869a339f571d63f&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c3&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c2&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c1&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1c0&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1bf&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
    {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f9870236416bbf6a2&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1be&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1bd&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1bc&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1bb&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ba&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722e7869a339f571d641&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b7&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f9870236416bbf6a4&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b4&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b3&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b2&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b1&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1b0&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f7869a339f571d643&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1af&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ae&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ad&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ac&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f9870236416bbf6a6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1a5&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1a6&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1a7&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1a8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1a9&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1aa&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf1ab&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f7869a339f571d645&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf199&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf198&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf197&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf196&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f9870236416bbf6a8&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf195&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf194&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf193&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf192&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f7869a339f571d647&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf191&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf190&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18f&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18e&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18d&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=643672309870236416bbf6aa&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18c&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18b&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf18a&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf189&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf188&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6436722f7869a339f571d649&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf187&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf186&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf185&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf184&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf183&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    },
     {
        "rml_url": "https://china.scidb.cn/download?fileId=6437f9c27869a339f571d95e&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
        "edf_urls": ["https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf16b&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf16a&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf169&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957",
                     "https://china.scidb.cn/download?fileId=61a0ca3f89f14b48842cf168&traceId=0ec4c131-82e0-4d7c-8183-b572a57f0957"]
    }

    # Add more patient groups as needed
]
# --- Set the base directory for all downloaded and generated files ---
base_dir = "/content/apnea_data"  # Using Colab's content directory
TARGET_SAMPLE_RATE = 256 # Hz


### Helper Functions
These cells define all the functions used in the pipeline. There is no need to edit them.

In [7]:
# === 3. FUNCTION DEFINITIONS ===

def download_file(url, out_path):
    """Downloads a file from a URL to a specified path using requests."""
    if not os.path.exists(out_path):
        print(f"Downloading {url}...")
        try:
            response = requests.get(url, stream=True, timeout=30)
            response.raise_for_status()  # Raise an exception for bad status codes
            with open(out_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Successfully downloaded to {out_path}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Failed to download {url}. Error: {e}")
            return False
    else:
        print(f"File already exists: {out_path}")
    return True

def extract_apnea_events(xml_file_path):
    """Parses an RML file and extracts start and end times of apnea events."""
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    namespace = {'ns': 'http://www.respironics.com/PatientStudy.xsd'}
    apnea_events = []
    apnea_types = ['ObstructiveApnea', 'CentralApnea', 'MixedApnea']
    for event in root.findall('.//ns:Event', namespace):
        if event.get('Family') == 'Respiratory' and event.get('Type') in apnea_types:
            start_time = float(event.get('Start'))
            duration = float(event.get('Duration'))
            end_time = start_time + duration
            apnea_events.append({'start': start_time, 'end': end_time})
    apnea_events.sort(key=lambda x: x['start'])
    print(f"Extracted {len(apnea_events)} apnea events from {os.path.basename(xml_file_path)}")
    return apnea_events

def is_apnea(frame_start, frame_end, events):
    """Checks if a given time frame overlaps with any apnea event."""
    for event in events:
        if frame_end > event['start'] and frame_start < event['end']:
            return 1  # Apnea
    return 0  # Normal

def extract_features(frame, sr):
    """Calculates a rich set of audio features from a single audio frame."""
    if np.all(frame == 0):
        return [0.0] * 23 # Return zeros for a silent frame

    # Basic features
    energy = np.mean(np.abs(frame))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=frame)[0])
    rms = np.mean(librosa.feature.rms(y=frame)[0])

    # Spectral features
    spectrum = np.abs(np.fft.rfft(frame))
    freqs = np.fft.rfftfreq(len(frame), 1/sr)
    centroid = librosa.feature.spectral_centroid(y=frame, sr=sr)[0, 0]
    bandwidth = librosa.feature.spectral_bandwidth(y=frame, sr=sr)[0, 0]
    rolloff = librosa.feature.spectral_rolloff(y=frame, sr=sr)[0, 0]
    flatness = librosa.feature.spectral_flatness(y=frame)[0, 0]

    # Advanced features
    mfccs = librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)
    skew = scipy.stats.skew(frame)
    kurt = scipy.stats.kurtosis(frame)

    # Spectrogram entropy
    power_spec = spectrum ** 2
    ps_norm = power_spec / (np.sum(power_spec) + 1e-10)
    entropy = -np.sum(ps_norm * np.log2(ps_norm + 1e-10))

    # Aggregate all features into a single list
    features = [energy, zcr, centroid, rms, bandwidth, rolloff, flatness, skew, kurt, entropy] + list(mfccs_mean)
    return features

In [8]:
# === 4. MASTER PROCESSING FUNCTION (UPDATED FOR DOWNSAMPLING) ===

def process_patient(patient_id, patient_group, base_dir, master_csv_path):
    """Runs the full data preparation pipeline for a single patient."""
    print(f"\n{'='*20} Processing {patient_id} {'='*20}")

    # Create a dedicated directory for the patient's files
    patient_dir = os.path.join(base_dir, patient_id)
    os.makedirs(patient_dir, exist_ok=True)

    # --- Step 1: Download all files for the patient ---
    print("\n--- Step 1: Downloading Files ---")
    rml_path = os.path.join(patient_dir, f"{patient_id}.rml")
    if not download_file(patient_group["rml_url"], rml_path):
        print(f"Skipping {patient_id} due to RML download failure.")
        return

    edf_paths = []
    for i, edf_url in enumerate(patient_group["edf_urls"], 1):
        edf_path = os.path.join(patient_dir, f"{patient_id}_part_{i}.edf")
        if download_file(edf_url, edf_path):
            edf_paths.append(edf_path)

    # --- Step 2: Concatenate EDF 'Mic' channels with chunking AND DOWNSAMPLING ---
    print(f"\n--- Step 2: Concatenating Audio to {TARGET_SAMPLE_RATE} Hz (Memory-Safe)---")
    output_wav = os.path.join(patient_dir, f"{patient_id}_full_mic.wav")

    # Validation loop to check original sample rates for consistency
    original_sfreq = None
    valid_edf_paths = []
    for edf_path in edf_paths:
        try:
            raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
            if "Mic" in raw.ch_names:
                current_sfreq = int(raw.info["sfreq"])
                if original_sfreq is None:
                    original_sfreq = current_sfreq
                elif original_sfreq != current_sfreq:
                    print(f"❌ CRITICAL ERROR: Sample rate mismatch in {os.path.basename(edf_path)}. Expected {original_sfreq}, found {current_sfreq}.")
                    return
                valid_edf_paths.append(edf_path)
            else:
                print(f"⚠️ WARNING: 'Mic' channel not found in {os.path.basename(edf_path)}. It will be skipped.")
        except Exception as e:
            print(f"❌ ERROR: Could not read {os.path.basename(edf_path)}. Error: {e}")

    if original_sfreq is None:
        print(f"❌ No valid EDF file with a 'Mic' channel found for {patient_id}. Skipping.")
        return

    # Memory-safe concatenation and downsampling loop
    with sf.SoundFile(output_wav, 'w', samplerate=TARGET_SAMPLE_RATE, channels=1, subtype='PCM_16') as out_f: # <-- CHANGE HERE
        for edf_path in valid_edf_paths:
            print(f"Processing in chunks: {os.path.basename(edf_path)}")
            raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)

            n_samples = raw.n_times
            chunk_size = original_sfreq * 600  # Read in 10-minute chunks based on ORIGINAL sample rate

            for start in range(0, n_samples, chunk_size):
                stop = min(start + chunk_size, n_samples)
                mic_chunk = raw.get_data(picks=["Mic"], start=start, stop=stop)[0].astype(np.float32)

                # --- THIS IS THE NEW DOWNSAMPLING LOGIC ---
                if original_sfreq != TARGET_SAMPLE_RATE:
                    mic_chunk = librosa.resample(y=mic_chunk, orig_sr=original_sfreq, target_sr=TARGET_SAMPLE_RATE)
                # --------------------------------------------

                out_f.write(mic_chunk)

    print(f"✅ Successfully created downsampled WAV: {os.path.basename(output_wav)}")

    # --- Step 3: Extract Features (this part remains unchanged) ---
    print("\n--- Step 3: Extracting Features and Labels ---")
    # ... The rest of the function continues exactly as before ...
    apnea_events = extract_apnea_events(rml_path)

    header = ['patient_id', 'frame_start', 'frame_end', 'energy', 'zcr', 'centroid', 'rms', 'bandwidth', 'rolloff', 'flatness', 'skew', 'kurt', 'entropy'] + [f'mfcc_{i+1}' for i in range(13)] + ['label']
    write_header = not os.path.exists(master_csv_path) or os.stat(master_csv_path).st_size == 0

    try:
        with sf.SoundFile(output_wav, 'r') as f_in:
            sr = f_in.samplerate # This will now be your TARGET_SAMPLE_RATE
            frame_sec = 1
            frame_len = int(sr * frame_sec)
            n_frames = int(np.floor(len(f_in) / frame_len))

            with open(master_csv_path, 'a', newline='') as f_out:
                writer = csv.writer(f_out)
                if write_header:
                    writer.writerow(header)

                for i in range(n_frames):
                    frame = f_in.read(frames=frame_len, dtype='float32')
                    if len(frame) < frame_len:
                        break

                    frame_start_sec = i * frame_sec
                    frame_end_sec = frame_start_sec + frame_sec

                    features = extract_features(frame, sr)
                    label = is_apnea(frame_start_sec, frame_end_sec, apnea_events)

                    writer.writerow([patient_id, frame_start_sec, frame_end_sec] + features + [label])
        print(f"✅ Appended {n_frames} frames from {patient_id} to master CSV.")
    except Exception as e:
        print(f"❌ ERROR during feature extraction for {patient_id}. Error: {e}")

In [9]:
# === 5. MAIN EXECUTION LOOP ===
# This cell runs the entire pipeline for all patients defined in the configuration.

master_csv_file = os.path.join(base_dir, "master_apnea_dataset.csv")

# Ensure the base directory exists
os.makedirs(base_dir, exist_ok=True)

for i, group in enumerate(patient_groups, 1):
    p_id = f"patient_{i}"
    process_patient(p_id, group, base_dir, master_csv_file)

print(f"\n{'='*20} PIPELINE COMPLETE {'='*20}")
print(f"Master dataset saved to: {master_csv_file}")


==================== Processing patient_1 ====================

--- Step 1: Downloading Files ---
Successfully downloaded to /content/apnea_data/patient_1/patient_1.rml
Successfully downloaded to /content/apnea_data/patient_1/patient_1_part_1.edf
Successfully downloaded to /content/apnea_data/patient_1/patient_1_part_2.edf
Successfully downloaded to /content/apnea_data/patient_1/patient_1_part_3.edf
Successfully downloaded to /content/apnea_data/patient_1/patient_1_part_4.edf
Successfully downloaded to /content/apnea_data/patient_1/patient_1_part_5.edf

--- Step 2: Concatenating Audio to 256 Hz (Memory-Safe)---


/tmp/ipython-input-8-1077296292.py:33: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:33: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:33: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:33: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:33: RuntimeWarning: Channels contain different highpass filters. Highest filter setti

Processing in chunks: patient_1_part_1.edf
Processing in chunks: patient_1_part_2.edf


/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


Processing in chunks: patient_1_part_3.edf


...

Processing in chunks: patient_20_part_3.edf


/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


Processing in chunks: patient_20_part_4.edf


/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
/tmp/ipython-input-8-1077296292.py:55: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)


✅ Successfully created downsampled WAV: patient_20_full_mic.wav

--- Step 3: Extracting Features and Labels ---
Extracted 201 apnea events from patient_20.rml
✅ Appended 12802 frames from patient_20 to master CSV.

==================== PIPELINE COMPLETE ====================
Master dataset saved to: /content/apnea_data/master_apnea_dataset.csv
